# Práctica 2: Transformaciones geométricas (Obligatorio – 5 puntos) 
- 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente. Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y. Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro. Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala. 

In [1]:
import cv2 as cv
import numpy as np
import tkinter as tk

# Cargar imagen
img = cv.imread('images/eii.png')
(h, w) = img.shape[:2]

# Definir el tamaño del lienzo de visualización.
# Esto asegura que la ventana siempre tenga el mismo tamaño.
import tkinter as tk

root = tk.Tk()
screen_w = root.winfo_screenwidth()
screen_h = root.winfo_screenheight()
root.destroy()

canvas_w, canvas_h = screen_w, screen_h

# Crear ventana con trackbars
cv.namedWindow("Transformaciones")

# Función vacía para trackbars
def nothing(x):
    pass

# Trackbars
cv.createTrackbar("Traslacion X", "Transformaciones", canvas_w // 2, canvas_w, nothing)
cv.createTrackbar("Traslacion Y", "Transformaciones", canvas_h // 2, canvas_h, nothing)
cv.createTrackbar("Angulo de Rotacion", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Centro X", "Transformaciones", w // 2, w, nothing)
cv.createTrackbar("Centro Y", "Transformaciones", h // 2, h, nothing)
cv.createTrackbar("Escala X", "Transformaciones", 100, 300, nothing)
cv.createTrackbar("Escala Y", "Transformaciones", 100, 300, nothing)
cv.createTrackbar("Escalado uniforme [0/1]", "Transformaciones", 0, 1, nothing)
cv.createTrackbar("Escala uniforme", "Transformaciones", 100, 300, nothing)

while True:
    # Leer valores de trackbars
    tx_track = cv.getTrackbarPos("Traslacion X", "Transformaciones")
    ty_track = cv.getTrackbarPos("Traslacion Y", "Transformaciones")
    angle = cv.getTrackbarPos("Angulo de Rotacion", "Transformaciones")
    cx = cv.getTrackbarPos("Centro X", "Transformaciones")
    cy = cv.getTrackbarPos("Centro Y", "Transformaciones")

    modo_uniforme = cv.getTrackbarPos("Escalado uniforme [0/1]", "Transformaciones")

    if modo_uniforme:
        escala_uniforme = max(cv.getTrackbarPos("Escala uniforme", "Transformaciones"), 1) / 100
        sx = escala_uniforme
        sy = escala_uniforme
    else:
        sx = max(cv.getTrackbarPos("Escala X", "Transformaciones"), 1) / 100
        sy = max(cv.getTrackbarPos("Escala Y", "Transformaciones"), 1) / 100

    # ----------------------------------------------------
    # FIX: Crear un lienzo o 'canvas' de tamaño fijo.
    canvas = np.zeros((canvas_h, canvas_w, 3), dtype=np.uint8)

    # Aplicar escalado a la imagen original.
    scaled_img = cv.resize(img, None, fx=sx, fy=sy, interpolation=cv.INTER_LINEAR)
    (sh, sw) = scaled_img.shape[:2]

    # Aplicar rotación y traslación a la imagen escalada.
    # El centro de rotación se aplica sobre la imagen original.
    M_rotate = cv.getRotationMatrix2D((cx * sx, cy * sy), angle, 1.0)
    rotated_img = cv.warpAffine(scaled_img, M_rotate, (sw, sh))

    # Calcular la traslación para centrar la imagen en el lienzo.
    tx = tx_track - sw // 2
    ty = ty_track - sh // 2

    # Calcular las coordenadas de la imagen rotada en el lienzo.
    x1, y1 = max(tx, 0), max(ty, 0)
    x2, y2 = min(tx + sw, canvas_w), min(ty + sh, canvas_h)

    # Calcular las coordenadas de la imagen rotada para cortar si se sale del lienzo.
    img_x1, img_y1 = max(-tx, 0), max(-ty, 0)
    img_x2, img_y2 = img_x1 + (x2 - x1), img_y1 + (y2 - y1)

    # Copiar la imagen transformada al lienzo.
    canvas[y1:y2, x1:x2] = rotated_img[img_y1:img_y2, img_x1:img_x2]

    # ----------------------------------------------------

    # Mostrar el lienzo final con la imagen dentro.
    cv.imshow("Transformaciones", canvas)

    if cv.waitKey(1) & 0xFF == 27: # Esc para salir
        break

cv.destroyAllWindows()

- 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen. 

In [ ]:
import cv2 as cv
import numpy as np

src = cv.imread('images/eii.png')   # Cargar imagen original
h_src, w_src = src.shape[:2]        # Altura y ancho

# Canvas donde se proyectará
canvas_h, canvas_w = 600, 900   
canvas_base = np.full((canvas_h, canvas_w, 3), 200, dtype=np.uint8)

# Variables globales
dst_pts = []                    # Puntos seleccionados
canvas = canvas_base.copy()     # Copia del canvas para dibujar

# Callback para clics
def seleccionar_punto(event, x, y, flags, param):
    global dst_pts, canvas
    if event == cv.EVENT_LBUTTONDOWN and len(dst_pts) < 4:
        dst_pts.append((x, y))
        cv.circle(canvas, (x, y), 5, (0, 0, 255), -1)
        if len(dst_pts) > 1:
            cv.line(canvas, dst_pts[-2], dst_pts[-1], (255, 0, 0), 2)
        if len(dst_pts) == 4:
            cv.line(canvas, dst_pts[-1], dst_pts[0], (255, 0, 0), 2)

# Ventanas iniciales
cv.imshow("Imagen original", src)
cv.namedWindow("Selecciona 4 puntos")
cv.setMouseCallback("Selecciona 4 puntos", seleccionar_punto)

while True:
    cv.imshow("Selecciona 4 puntos", canvas)
    key = cv.waitKey(1) & 0xFF

    if key == 27:  # ESC → salir
        break

    elif key == ord('r'):  # tecla R → reiniciar selección
        dst_pts = []
        canvas = canvas_base.copy()

    elif key == 13 and len(dst_pts) == 4:  # ENTER → proyectar
        dst_pts_np = np.array(dst_pts, dtype=np.float32)
        src_pts = np.array([[0, 0], [w_src, 0], [w_src, h_src], [0, h_src]], dtype=np.float32)

        H, _ = cv.findHomography(src_pts, dst_pts_np)
        warped = cv.warpPerspective(src, H, (canvas_w, canvas_h))

        # Máscara y mezcla
        mask = np.zeros((canvas_h, canvas_w), dtype=np.uint8)
        cv.fillConvexPoly(mask, dst_pts_np.astype(np.int32), 255)
        mask3 = cv.merge([mask, mask, mask])
        result = np.where(mask3 == 255, warped, canvas_base)

        cv.imshow("Imagen proyectada", result)

cv.destroyAllWindows()


In [12]:
import cv2 as cv
import numpy as np

# Cargar imagen
img = cv.imread('images/eii.png')
(h, w) = img.shape[:2]

# Crear ventana
cv.namedWindow("Proyeccion")

# Función vacía
def nothing(x):
    pass

# Trackbars para definir la ventana de proyección
cv.createTrackbar("X", "Proyeccion", 0, w - 1, nothing)
cv.createTrackbar("Y", "Proyeccion", 0, h - 1, nothing)
cv.createTrackbar("Ancho", "Proyeccion", w // 4, w, nothing)
cv.createTrackbar("Alto", "Proyeccion", h // 4, h, nothing)

while True:
    # Leer valores de trackbars
    x = cv.getTrackbarPos("X", "Proyeccion")
    y = cv.getTrackbarPos("Y", "Proyeccion")
    win_w = cv.getTrackbarPos("Ancho", "Proyeccion")
    win_h = cv.getTrackbarPos("Alto", "Proyeccion")

    # Asegurar que la ventana no se salga de la imagen
    x = min(x, w - 1)
    y = min(y, h - 1)
    win_w = min(win_w, w - x)
    win_h = min(win_h, h - y)

    # Copia para mostrar rectángulo sobre la imagen
    img_with_rect = img.copy()
    cv.rectangle(img_with_rect, (x, y), (x + win_w, y + win_h), (0, 255, 0), 2)

    # Extraer ventana de proyección
    roi = img[y:y + win_h, x:x + win_w]

    # Proyección (escalada a tamaño fijo, por ejemplo 300x300)
    projection = cv.resize(roi, (300, 300), interpolation=cv.INTER_LINEAR)

    # Mostrar ambas imágenes
    cv.imshow("Proyeccion", img_with_rect)
    cv.imshow("Ventana Proyectada", projection)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()


- 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz 

In [2]:
import cv2
import numpy as np

def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]

    # Coeficientes de distorsión
    distCoeff = np.zeros((4, 1), np.float64)
    distCoeff[0, 0] = k1  # Término radial de primer orden
    distCoeff[1, 0] = k2  # Término radial de segundo orden

    # Matriz de cámara ficticia
    cam = np.eye(3, dtype=np.float32)
    cam[0, 2] = w / 2.0  # Centro x
    cam[1, 2] = h / 2.0  # Centro y
    cam[0, 0] = 10.0     # Focal x
    cam[1, 1] = 10.0     # Focal y

    # Aplicar distorsión
    distorted_img = cv2.undistort(image, cam, distCoeff)

    return distorted_img

# Cargar imagen
image = cv2.imread('images/eii.png')
if image is None:
    print("No se pudo cargar la imagen.")
    exit()

# Crear ventana y trackbars
cv2.namedWindow("Distorsión Interactiva")

def nothing(x):
    pass

# Trackbars con rango -100 a 100 → convertir a rango -0.1 a 0.1
cv2.createTrackbar("k1", "Distorsión Interactiva", 100, 200, nothing)
cv2.createTrackbar("k2", "Distorsión Interactiva", 100, 200, nothing)

while True:
    # Leer valores de trackbars
    k1_raw = cv2.getTrackbarPos("k1", "Distorsión Interactiva")
    k2_raw = cv2.getTrackbarPos("k2", "Distorsión Interactiva")

    # Escalar a [-0.1, 0.1]
    k1 = (k1_raw - 100) / 100000.0
    k2 = (k2_raw - 100) / 100000.0

    # Aplicar distorsión
    distorted = apply_distortion(image, k1, k2)

    # Mostrar imagen
    cv2.imshow("Distorsión Interactiva", distorted)

    # Salir con ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()


- Marcar el punto de giro con el ratón.
- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.
- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en
una imagen destino y realizar la transformación afín.
- Calcular la imagen especular a partir de una imagen.
- Tratar una recta que será el eje de reflexión y “reflejar” la imagen.
- Otras aportaciones

In [7]:
import cv2 as cv
import numpy as np

# Carga
src = cv.imread('images/eii.png')
h, w = src.shape[:2]

# Estado
offset = np.array([0,0], dtype=np.int32)
dragging = False
start_mouse = (0,0)
start_offset = np.array([0,0], dtype=np.int32)
rotation_center = np.array([w//2, h//2], dtype=np.float32)  # por defecto centro
angle = 0  # ángulo inicial

# Canvas donde mostrar (tamaño ajustable)
canvas_h, canvas_w = max(600, h+200), max(900, w+400)
canvas_base = np.full((canvas_h, canvas_w, 3), 200, dtype=np.uint8)

def nothing(x):
    pass

def draw_scene():
    # aplicar rotación antes de dibujar
    M = cv.getRotationMatrix2D(tuple(rotation_center), angle, 1.0)
    rotated = cv.warpAffine(src, M, (w, h), borderValue=(200,200,200))

    canvas = canvas_base.copy()
    # coordenada superior izquierda donde se dibuja la imagen
    tx, ty = offset
    # recortar por zonas fuera de canvas:
    x1, y1 = max(0, tx), max(0, ty)
    x2, y2 = min(canvas_w, tx + w), min(canvas_h, ty + h)
    sx1, sy1 = x1 - tx, y1 - ty
    sx2, sy2 = sx1 + (x2 - x1), sy1 + (y2 - y1)
    if x1 < x2 and y1 < y2:
        canvas[y1:y2, x1:x2] = rotated[sy1:sy2, sx1:sx2]

    # dibujar punto de giro (en coordenadas globales)
    cx, cy = int(rotation_center[0] + tx), int(rotation_center[1] + ty)
    cv.circle(canvas, (cx, cy), 6, (0,0,255), -1)

    cv.putText(canvas, f"Angulo: {angle} grados", (10,20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2)
    cv.putText(canvas, "Clic izq+arrastrar = mover | Clic der = cambiar centro giro", (10,40),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv.putText(canvas, "r = reset | ESC = salir", (10,60),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    return canvas

def mouse_cb(event, x, y, flags, param):
    global dragging, start_mouse, start_offset, offset, rotation_center
    if event == cv.EVENT_LBUTTONDOWN:
        dragging = True
        start_mouse = (x, y)
        start_offset = offset.copy()
    elif event == cv.EVENT_MOUSEMOVE and dragging:
        dx = x - start_mouse[0]
        dy = y - start_mouse[1]
        offset = start_offset + np.array([dx, dy])
    elif event == cv.EVENT_LBUTTONUP:
        dragging = False
    elif event == cv.EVENT_RBUTTONDOWN:
        # establecer punto de giro relativo a la imagen (si el click cae sobre la imagen)
        tx, ty = offset
        rx, ry = x - tx, y - ty
        if 0 <= rx < w and 0 <= ry < h:
            rotation_center = np.array([rx, ry], dtype=np.float32)

cv.namedWindow("Interactive transform")
cv.setMouseCallback("Interactive transform", mouse_cb)
cv.createTrackbar("Angulo", "Interactive transform", 0, 360, nothing)

while True:
    # leer ángulo desde trackbar
    angle = cv.getTrackbarPos("Angulo", "Interactive transform")

    canvas = draw_scene()
    cv.imshow("Interactive transform", canvas)

    k = cv.waitKey(20) & 0xFF
    if k == 27:  # ESC
        break
    elif k == ord('r'):  # reset
        offset = np.array([0,0], dtype=np.int32)
        rotation_center = np.array([w//2, h//2], dtype=np.float32)
        cv.setTrackbarPos("Angulo", "Interactive transform", 0)

cv.destroyAllWindows()
